# Desafio (Parte 2) - Extraindo Informações

## Case baseado no desafio do Kaggle: "Instacart Market Basket Analysis"

Esse desafio é baseado em uma competição do Kaggle ([https://www.kaggle.com](https://www.kaggle.com)) de 2017, onde é pedido para fazer a predição de se um produto será comprado por dado usuário no carrinho atual, baseado em suas compras anteriores.

Não está no escopo da aula 



Link para o desafio: [https://www.kaggle.com/c/instacart-market-basket-analysis](https://www.kaggle.com/c/instacart-market-basket-analysis)

## Origem dos Dados:

Os dados mostrados nesse desafio são uma amostra extraída do dataset disponibilizado gratuitamente pela empresa Instacart ([https://www.instacart.com/](https://www.instacart.com/)) **\***. São dados reais do banco de dados da empresa, que foram anonimizados para o uso por pesquisadores e em competições de Data Science.

 
###### * “The Instacart Online Grocery Shopping Dataset 2017”, Accessed from [https://www.instacart.com/datasets/grocery-shopping-2017](https://www.instacart.com/datasets/grocery-shopping-2017) on March 10, 2017.

## Tabelas e Colunas Importantes

As Tabelas e suas respectivas colunas estão mostradas abaixo, junto com a descrição do significado de cada coluna:

* Colunas da Tabela `orders`:

    * ´order_id´: order identifier
    * ´user_id´: customer identifier
    * ´eval_set´: which evaluation set this order belongs in (see SET described below)
    * ´order_number´: the order sequence number for this user (1 = first, n = nth)
    * ´order_dow´: the day of the week the order was placed on
    * ´order_hour_of_day´: the hour of the day the order was placed on
    * ´days_since_prior´: days since the last order, capped at 30 (with NAs for order_number = 1)
    
* Colunas da Tabela `products (50k rows):

    * ´product_id´: product identifier
    * ´product_name´: name of the product
    * ´aisle_id´: foreign key
    * ´department_id´: foreign key

* Colunas da Tabela `aisles (134 rows):

    * ´aisle_id´: aisle identifier
    * ´aisle´: the name of the aisle

* Colunas da Tabela `deptartments (21 rows):

    * ´department_id´: department identifier
    * ´department´: the name of the department

* Colunas da Tabela `order_products__SET (30m+ rows):

    * ´order_id´: foreign key
    * ´product_id´: foreign key
    * ´add_to_cart_order´: order in which each product was added to cart
    * ´reordered´: 1 if this product has been ordered by this user in the past, 0 otherwise

___

# Import

In [2]:
import os
import pandas as pd

# Objetivo:

Analisar o dataset gerado na Seção 3 e responder às questões.

# Dataset

In [3]:
file_name = os.path.join("data", "dataset.csv")

In [25]:
dataset = pd.read_csv(file_name,
                      sep=",",
                      encoding="utf-8")

In [26]:
""" Facilitando a vida """
# baseado em uma thread do Kaggle sobre esse desafio, os dias da semana são definidos na seguinte ordem:
week_days = ["saturday", "sunday", "monday", "tuesday", "wednesday", "thursday", "friday"]

# traduzindo no dataset
dataset.order_dow = dataset.order_dow.map(lambda wd: week_days[wd])

In [27]:
dataset.head()

,order_id,user_id,order_dow,department,aisle,product_name
0,1187899,1,wednesday,beverages,soft drinks,Soda
1,2757217,67,saturday,beverages,soft drinks,Soda
2,632715,676,saturday,beverages,soft drinks,Soda
3,1167274,760,wednesday,beverages,soft drinks,Soda
4,3347074,804,tuesday,beverages,soft drinks,Soda


# Questões 

## A) Quantos usuários únicos fizeram pedidos?

In [28]:
unique_users = dataset['user_id'].nunique()
print(unique_users)

117221


## B) Qual o produto que apareceu em mais pedidos?

In [57]:
top_products = dataset['product_name'].value_counts().head(5)
top_products

Banana                    16784
Bag of Organic Bananas    13790
Organic Strawberries       9740
Organic Baby Spinach       8734
Large Lemon                7254
Name: product_name, dtype: int64

## C) Quais os usuários que fizeram mais e menos pedidos?

In [84]:
df_orders = dataset[['user_id','order_id','product_name']].groupby(['user_id','order_id']).count()
df_orders = df_orders.reset_index(level=['user_id','order_id'])
df_orders.head()

,user_id,order_id,product_name
0,1,1187899,11
1,2,1492625,31
2,5,2196797,9
3,7,525192,9
4,8,880375,18


In [89]:
df_users = df_orders[['user_id','order_id']].groupby(['user_id']).count()
df_users = df_users.reset_index(level=['user_id'])
df_users.head(5).append(df_users.tail(5))

,user_id,order_id
0,1,1
1,2,1
2,5,1
3,7,1
4,8,1
117216,184273,1
117217,184274,1
117218,184277,1
117219,184278,1
117220,184279,1


## D) Qual a média e o desvio padrão da quantidade de produtos presentes em um pedido um usuário?

In [31]:
groupped_columns = ['order_id','user_id','order_dow']
df = dataset[['product_name'] + groupped_columns].groupby(groupped_columns).agg('count')
df = df.reset_index()
df.rename(columns={'product_name':'product_qty'},
          inplace=True)
df.head()

,order_id,user_id,order_dow,product_qty
0,1,112108,wednesday,8
1,36,79431,friday,8
2,38,42756,friday,9
3,96,17227,friday,7
4,98,56463,tuesday,49


In [32]:
product_qty_avg = df['product_qty'].mean()
product_qty_stddev = df['product_qty'].std()
print('Average: ', product_qty_avg)
print('Standard Derivation: ', product_qty_stddev)

Average:  10.564190716680459
Standard Derivation:  7.945046845800925


## E) Qual a média, a mediana, o mínimo e o máximo da quantidade de corredores (`aisles`) por departmento (`department`)?


In [33]:
df2 = dataset[['department','aisle']].groupby(['department']).agg('count')
df2 = df2.reset_index()
df2.rename(columns={'aisle':'aisle_qty'},
          inplace=True)
df2.head()

,department,aisle_qty
0,alcohol,5007
1,babies,13324
2,bakery,43259
3,beverages,102005
4,breakfast,26456


In [34]:
print(df2['aisle_qty'].mean())
print(df2['aisle_qty'].median())
print(df2['aisle_qty'].min())
print(df2['aisle_qty'].max())

58968.80952380953
32299.0
1218
365640


## F) Qual o dia da semana em que ocorrem mais pedidos de produtos para cada departmento (`department`)?

In [70]:
df3 = dataset[['department','order_dow','order_id']]
df3 = df3.groupby(['department','order_dow']).agg('count')
df3.rename(
    columns={'order_id':'order_qty'},
    inplace=True
)
df3 = df3.reset_index(level=['department', 'order_dow'])
df3.head(10)

,department,order_dow,order_qty
0,alcohol,friday,683
1,alcohol,monday,578
2,alcohol,saturday,640
3,alcohol,sunday,571
4,alcohol,thursday,929
5,alcohol,tuesday,753
6,alcohol,wednesday,853
7,babies,friday,1769
8,babies,monday,1637
9,babies,saturday,3143


In [76]:
df4 = df3[['department', 'order_qty']].groupby(['department']).max()
df4 = df4.reset_index(level=['department'])
df4.head()

,department,order_qty
0,alcohol,929
1,babies,3143
2,bakery,10088
3,beverages,20249
4,breakfast,5936


In [92]:
result = pd.merge(
    df3,
    df4,
    how='left',
    left_on=['department','order_qty'],
    right_on=['department','order_qty']
)
result.head()

,department,order_dow,order_qty
0,alcohol,friday,683
1,alcohol,monday,578
2,alcohol,saturday,640
3,alcohol,sunday,571
4,alcohol,thursday,929


##  G) Quais foram os 5% piores e os 5% melhores corredores (`aisle`) em termos de vendas? 

In [ ]:
""" Entre com o Código aqui """

##  H) Quais foram os 5 produtos mais comprados de cada departamento na terça-feira? 

### Mostre em uma tabela como no exemplo:

| departamento | 1o produto | 2o produto | 3o produto | 4o produto | 5o produto |
|:------------ |:---------- |:---------- |:---------- |:---------- |:---------- |
| depart 01    | prod A     | prod W     | prod L     | prod H     | prod D     |
| depart 02    | prod X     | prod Q     | prod B     | prod S     | prod P     |
| ...          | ...        | ...        | ...        | ...        | ...        |


In [ ]:
""" Entre com o Código aqui """

## I) Quais são os 2 corredores (`aisles`) que são visitados juntos no mesmo pedido mais vezes em uma sexta-feira?

In [ ]:
""" Entre com o Código aqui """

## J) Quais as duplas de produtos que mais saem juntas no pedido (`order_id`) de um usuário em uma sexta-feira?

In [ ]:
""" Entre com o Código aqui """